# Sample for Kubeflow PyTorchJob SDK

This is a sample for Kubeflow PyTorchJob SDK `kubeflow-pytorchjob`.

The notebook shows how to use Kubeflow PyTorchJob SDK to create, get, update and delete PyTorchJob.

In [ ]:
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container
from kubernetes.client import V1ResourceRequirements

from kubeflow.pytorchjob import constants
from kubeflow.pytorchjob import utils
from kubeflow.pytorchjob import V1ReplicaSpec
from kubeflow.pytorchjob import V1PyTorchJob
from kubeflow.pytorchjob import V1PyTorchJobSpec
from kubeflow.pytorchjob import PyTorchJobClient

Define namespace where pytorchjob needs to be created to. If not specified, below function defines namespace to the current one where SDK is running in the cluster, otherwise it will deploy to default namespace.

In [ ]:
namespace = utils.get_default_target_namespace()

### Define PyTorchJob

The demo only creates a worker of PyTorchJob to run mnist sample.

In [ ]:
container = V1Container(
    name="pytorch",
    image="gcr.io/kubeflow-ci/pytorch-dist-mnist-test:v1.0",
    args=["--backend","gloo"]
)

master = V1ReplicaSpec(
    replicas=1,
    restart_policy="OnFailure",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)


pytorchjob = V1PyTorchJob(
    api_version="kubeflow.org/v1",
    kind="PyTorchJob",
    metadata=V1ObjectMeta(name="pytorch-dist-mnist-gloo",namespace=namespace),
    spec=V1PyTorchJobSpec(
        clean_pod_policy="None",
        pytorch_replica_specs={"Master": master}
    )
)


### Create PyTorchJob

In [ ]:
pytorch_client = PyTorchJobClient()
pytorch_client.create(pytorchjob)

### Get the created PyTorchJob 

In [ ]:
pytorch_client.get('pytorch-dist-mnist-gloo')

### Update the created PyTorchJob

In [ ]:
worker = V1ReplicaSpec(
    replicas=1,
    restart_policy="OnFailure",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

pytorchjob = V1PyTorchJob(
    api_version="kubeflow.org/v1",
    kind="PyTorchJob",
    metadata=V1ObjectMeta(name="pytorch-dist-mnist-gloo",namespace=namespace),
    spec=V1PyTorchJobSpec(
        clean_pod_policy="None",
        pytorch_replica_specs={"Master": master,
                               "Worker": worker}
    )
)

pytorch_client.patch('pytorch-dist-mnist-gloo', pytorchjob)

### Delete the PyTorchJob

In [ ]:
pytorch_client.delete('pytorch-dist-mnist-gloo')